In [1]:
# Descargamos las librerias que necesitamos

import requests # Para hacer request
import re # Regex
from bs4 import BeautifulSoup # Para hacer la sopa del request

# Manejo de datos 

import pandas as pd # manejo de datos y dataframes
import numpy as np # manejo de arrays y operaciones matematicas 


# Librerias para realizar web scraping con selenium

from selenium import webdriver # webdriver permite manejar un navegador 
from webdriver_manager.chrome import ChromeDriverManager # permite instalar y mantener actualizado el driver de chrome
from selenium.webdriver.common.keys import Keys # permite simular teclas del teclado 
from selenium.webdriver.chrome.options import Options # permite configurar el driver de chrome como modo incognito o maximizar la ventana
from time import sleep # esperas entre ejecuciones de codigo


import warnings # para ignorar los warnings de python 
warnings.filterwarnings('ignore')

from tqdm import tqdm

In [2]:
from selenium.webdriver.support.ui import WebDriverWait # proporciona la capacidad de esperar de manera explícita hasta que se cumpla una determinada condición antes de continuar con la ejecución del código
from selenium.webdriver.support import expected_conditions as EC # define una serie de condiciones esperadas que se pueden utilizar con WebDriverWait. Estas condiciones representan eventos o estados específicos que se esperan que ocurran en el navegador web, como la presencia de un elemento en la página, su visibilidad, etc. 
from selenium.webdriver.common.by import By # proporciona una serie de estrategias de localización para encontrar elementos en una página web.

In [3]:
opciones= Options() # se utiliza para configurar las opciones del navegador.
opciones.add_experimental_option('excludeSwitches', ['enable-automation']) #  se utiliza para agregar una opción experimental al navegador. En este caso, se está configurando la opción "excludeSwitches" con el valor ['enable-automation']. 
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False) # configuramos la opción "useAutomationExtension" con el valor False. Esta opción controla si se debe utilizar la extensión de automatización del navegador. Al establecerlo en False, se deshabilita la extensión de automatización.
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

In [4]:
espectaculos = []
urls_shows = []
precios = []

for i in range(1, 7):
    url = f'https://www.atrapalo.com/entradas/madrid/teatro-y-danza/monologos/p-{i}/'
    
    # hacemos la request a la página
    res1 = requests.get(url)
    # creamos la sopa
    sopa = BeautifulSoup(res1.content, 'html.parser')


    # Sacamos NOMBRES e iteramos para añadirlos a una lista
    nombres = sopa.find_all("h2", {"class": "clear nombre"})
    for i in nombres:
        espectaculos.append(i.getText().strip())
    
    # Sacamos urls de los shows e iteramos para añadirlos a una lista
    urls = sopa.find_all("div", {"data-url": True})
    for url in urls:
        urls_shows.append(url['data-url'])
    
    # Sacamos los precios de los shows e iteramos para añadirlos a una lista    
    p = sopa.find_all("span", {"class": "value"})
    
    for i in p:
        precios.append(i.getText().strip())

In [5]:
monologos_madrid = {"nombre":[], 
                        "precio": [],
                         "artista": []}

# Introducimos los nombres de los shows al diccionario
    
monologos_madrid["nombre"] = espectaculos
    
# Introducimos los precios de los shows al diccionario

monologos_madrid["precio"] = precios

In [6]:
driver = webdriver.Chrome()
for url in tqdm(urls_shows):
    driver.get(f"https://www.atrapalo.com{url}") 
    driver.implicitly_wait(5)

    # aceptamos las cookies de trapalo
    try:
        driver.find_element("css selector","#CybotCookiebotDialogBodyButtonAccept").click()
        sleep(2)
    except:
        pass
    
    # Capturamos nombre del artista
    try:
        artistas_contenedor = driver.find_element("css selector", "div.artists")
        artistas_texto = artistas_contenedor.text
        artistas_texto = artistas_texto.split(":")[1].strip()  # Extraer lo que está después de ':'
        monologos_madrid["artista"].append(artistas_texto)
    except:
        monologos_madrid["artista"].append(np.nan)
driver.quit()  

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [25:48<00:00,  6.76s/it]


In [7]:
# Comprobamos la longitud de todas las llaves para comprobar que se ha descargado todos los datos con exito

print(len(monologos_madrid["nombre"]))
print(len(monologos_madrid["precio"]))
print(len(monologos_madrid["artista"]))

229
229
229


In [8]:
monologos_madrid["artista"]

[nan,
 nan,
 nan,
 'Victor Parrado',
 'Alex Salaberri Rubén Faura Marce González Raúl Fervé David Carras',
 'Galder Varas',
 'Diego Daño, Eva Soriano, Fran Pati, Nacho García, El Golfo Comedy Club',
 'Jorge Santos; Carmen Orellana; Patxi Zubeldia; Juan Moreno; Sara Drama; Edu J; Iván Perlaza; Javi Bueno',
 'Ignatius, Juan Solo, Nene, Luis Álvaro, Patricia Galván, Carolina Noriega, Coria Castillo, Miguel Miguel, Dani Alés, JJ Vaquero, Virginia Riezu, Jaime Caravaca, Javi Durán, Darío Mares, Silvia Sparks, Los Jackass de la Impro, Comikazes, Richar Salamanca, Galder Varas, Raúl Massana',
 'Teatro de las Aguas',
 nan,
 'Juan Aroca',
 'Miguel Lago',
 'Alex O’Dogherty',
 'La Chocita del Loro',
 'Joan Natzari',
 'Karim',
 'Pilar Ordóñez',
 'Jorge Santos',
 'Alex Salaberri',
 'Miguel Iribar, Quique Matilla, Dani Alés, Fernando García-Torres, Raúl Navareño, Fran El Chavo, Danny Boy-Rivera , Eva y qué, Alex Salaberri, Álvaro Casares, David Cepo, Archie Bezos, La Chocita del Loro',
 'Javi Delpe,

In [11]:
df_artistas_madrid = pd.DataFrame(monologos_madrid)
df_artistas_madrid

,nombre,precio,artista
0,La Hora y Media de El Club de la Comedia Madrid,22€,NaN
1,La Noche de la Risa,18€,NaN
2,"No cruces los brazos, David Cepo",17€,NaN
3,"Buenrollistas - Víctor Parrado, en Madrid",14€,Victor Parrado
4,Malasaña - Los Mejores Monólogos de Madrid,7€,Alex Salaberri Rubén Faura Marce González Raúl...
...,...,...,...
224,Micro Abierto Monólogos Madrid,7€,NaN
225,Diego Daño. Viviendo para Instagram,11€,"Diego Daño, El Golfo Comedy Club"
226,Más vale solo que ciento volando. Reloaded,18€,Edu Soto
227,"Raúl Massana ""Vendo Humo"" Stand up show",12€,Raúl Massana


In [ ]:
# Especificamos la ruta y el nombre del archivo CSV
ruta_archivo_csv = 'C:/Users/STF/Desktop/Ironhack/proyectos/ETL_project/01_data/artistas_madrid.csv'

# Guardamos el nuevo DataFrame en un archivo CSV
df_artistas_madrid.to_csv(ruta_archivo_csv, index=False)